# Spaced repetition

Spaced repetition is a learning technique with the goal of memorisation whereby flashcards are shown to the student at increasing intervals. The goal of this notebook is to discuss some possible approaches to spaced repetition.

## Possible approaches

### Machine Learning

My hunch is that machine learning would be a good framework for building a spaced repetition algorithm. In particular, this could be done using reinforcement learning in online mode. The question is, what would a good loss function be?

#### Loss function

Here are some ideas for a loss function:

- Mean interval length in the queue

The idea behind mean interval length in the queue is that we would like the model to encourage students to remember things for long periods of time. However, the problem is that the model can choose arbitrarily long intervals even if the student will surely fail once the word is presented to them.

## Getting started

To get started, we want to implement this model as a recommender system. The idea is to recommend a next card to review with the goal of increasing card retention by measuring the time-to-forget directly.

In order to get started, perhaps we should start by loading an anki deck into python.

## Loading an Anki Deck

In [2]:
import pandas as pd

In [3]:
from anki_export import ApkgReader

In [4]:
with ApkgReader('../data/Talk_to_Me_in_Korean_TTMIK_Supplement_-_2021_Update_Pt_2.apkg') as apkg:
    anki_data = apkg.export()

In [5]:
anki_data.keys()

odict_keys(['Korean Detailed Grammar (with reversed card)', 'Korean Detailed Vocabulary (with reversed card)', 'Cloze Hangul'])

In [9]:
anki_df = pd.DataFrame(anki_data['Korean Detailed Grammar (with reversed card)'])
anki_df.columns = anki_df.iloc[0]
anki_df = anki_df[1:]

In [11]:
anki_df

,Front,Back,Audio,"Notation (brackets, etc.)",Quick Notes,Alternate Phrasing,Rules,Rule Alternate Meanings,Other Rules with Similar Meanings,"Rule Used, but Unrelated to Primary",Irregular Rules,Phonetics,References,tags,deck
1,Hello.<div><br><div>How are you?</div></div>,안녕하세요.,[sound:TTMIK 1-1 01.mp3],,"The ending ""~세요"" is explained in TTMIK-2.26.&n...",,,,,,,<div>[an-nyeong ha-se-yo]</div>,,KGIU-07.01 TTMIK-1.01 TTMIK-2.26,TTMIK Supplement::TTMIK Level 1
2,"<div><sup><i><font color=""#7ad2e9"">[formal]</f...",감사합니다.,[sound:TTMIK 1.1 02.mp3],,,,,,,,,<div>[gam-sa hap-ni-da]</div>,,TTMIK-1.01,TTMIK Supplement::TTMIK Level 1
3,<div><sup><i>[general agreement]</i></sup></di...,네,[sound:TTMIK 1.2 01.mp3],,,,<u>Super broad meaning</u>:<div>• any type of ...,,,,,[ne],,TTMIK-1.02,TTMIK Supplement::TTMIK Level 1
4,"Yes, that’s right.","네, 맞아요.",[sound:TTMIK 1.02 02.mp3],,,,,,,,,"[ne, ma-ja-yo]",,TTMIK-1.02,TTMIK Supplement::TTMIK Level 1
5,<div>no</div><div><br></div><div>that's not ri...,아니요,[sound:TTMIK 1.02 03.mp3],,,,Nuanced: a word indicating disagreement with w...,,,,,<div>[a-ni-yo]</div>,,TTMIK-1.02,TTMIK Supplement::TTMIK Level 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,{You} are a liar.<div><br></div><div>{He} is a...,거짓말쟁이예요.,[sound:you are a liar.mp3],,,,,,"""a lie"" is ""거짓말""<div><br></div><div>""to lie"" i...",,,,,TTMIK-2.29,TTMIK Supplement::TTMIK Level 2
1109,<div>`All done.`<br></div><div><br></div><div>...,다 끝났어요,[sound:all done.mp3],,,,"<u><b><font color=""#ff0000"">다 (verb)</font></b...",,,,,,,TTMIK-2.29,TTMIK Supplement::TTMIK Level 2
1110,<i><sup>[subject marked]</sup></i><div><br></d...,엄마가 더 무서웠어요.,[sound:mom was scarier.mp3],,"""ㅂ"" irregular covered in TTMIK-3.11",,"<u><b><font color=""#ff0000"">더 (verb)</font></b...",,,,,,,TTMIK-2.30,TTMIK Supplement::TTMIK Level 2
1111,Don't tell mom.,엄마한테 말하지 마세요.,[sound:dont tell mom.mp3],,The example is one sibling talking to another....,,"<div><u><b><font color=""#ff0000"">(V stem)~지 마세...",,,,,,,TTMIK-2.30,TTMIK Supplement::TTMIK Level 2


Now that we can load the anki deck into python, the question is how to represent the user feedback.

We want to do something like the following:

1. Show the user a card
2. Let the user pick whether they know it or not
3. Add that to the user's history
4. Show the user a next card, with the goal of optimising memory retention

The question is, how do we quantify "memory retention" for the purpose of this application?

In principle, we want the model to keep track of how long the user remembers each card and help them
remember cards for as long as possible.